In [1]:
from cosecha_colectiva import miscelaneous as m
m.try_find_conf_file()

from cosecha_colectiva import main_tester as mt
from cosecha_colectiva import query_cosecha as qc
from cosecha_colectiva import config
from cosecha_colectiva import CAF_DB_Monitor as CDM
from cosecha_colectiva import test_cosecha_api as tca

import logging
import pathlib
from datetime import datetime
import pandas as pd

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


[2023-05-07 10:09:48,914][INFO]: Connecting b3cf7edd2f8490@us-cdbr-east-06.cleardb.net:3306
[2023-05-07 10:09:49,754][INFO]: Connected b3cf7edd2f8490@us-cdbr-east-06.cleardb.net:3306


In [2]:
#xls_name = 'CAF_GrupoTesteador.xlsx'
#xls_name = 'CAF_GrupoAstronomo.xlsx'
xls_name = 'CAF_GrupoDomador.xlsx'
#xls_name = 'CAF_GrupoMesero.xlsx'

In [3]:
id_grupo = qc.get_grupo_id_by_name(xls_name.split('.')[0])
print(id_grupo)

1214


In [4]:
qc.delete_grupo(id_grupo, solo_sesiones=True, force_delete=True)

([], 'transaccion_prestamos')
([], 'transacciones')
([], 'interes_prestamo')
([], 'ganancias')
([], 'multas')
([], 'prestamos')
([], 'prestamos')
([], 'asistencias')
([], 'sesiones')
([], 'acuerdos')


In [5]:
now = datetime.now()
log_date = now.strftime("%Y-%m-%d_%H_%M_%S")
log_file = pathlib.Path(config.log_dir, log_date+'_Sesiones.log')
logging.basicConfig(filename=log_file, encoding='utf-8', level=logging.INFO)

In [6]:
monitor_grupo = CDM.CAF_DB_Monitor(id_grupo)

[2023-05-07 10:15:58,298][WARNING]: MySQL server has gone away. Reconnecting to the server.


In [7]:
start_month = 1
end_month = 9

for month in range(start_month, end_month+1):

    print('month: ', month)

    try:
        monitor_grupo.actualiza_todo_sesion(xls_name, month, type_xls='NEW')
    except Exception as e:
        print(e)
        raise(e)
    try:
        mt.test_sesion_months(xls_name, month, month, type_xls='NEW')
    except Exception as e:
        print(e)
        raise(e)
    [status, dfs] = monitor_grupo.compara_db_monitor()
    if not status:
        print('Algo no coincidió')
        break

month:  1


AttributeError: 'CAF_DB_Monitor' object has no attribute 'interes_prestamos'

In [8]:
comp = dfs[0].compare(dfs[1])
comp

Monto_ganancia      
             self other
80            3.2  3.15
81            3.2  3.15
82            3.2  3.15
83            3.2  3.15
84            3.2  3.15
85            3.2  3.15
86            3.2  3.15
87            3.2  3.15
88            3.2  3.15
89            3.2  3.15

In [9]:
monitor_grupo.interes_prestamo

,Interes_prestamo_id,Prestamo_id,Sesion_id,Monto_interes,Tipo_interes
0,4014,2644,6314,10.0,0
1,4024,2644,6324,10.0,0
2,4034,2654,6324,8.0,0
3,4044,2644,6334,10.0,0
4,4054,2654,6334,8.0,0
5,4064,2644,6344,17.5,1
6,4074,2654,6344,8.0,0
7,4084,2644,6354,17.5,1
8,4094,2654,6354,8.0,0
9,4104,2644,6364,17.5,1


In [9]:
monitor_grupo.prestamos

,Prestamo_id,Monto_prestamo,Monto_pagado,Interes_generado,Interes_pagado,Estatus_ampliacion,Num_sesiones,Sesiones_restantes,Estatus_prestamo,Socio_id,...,Interes_morosidad,Ampliacion_prestamos,Interes_ampliacion,Mod_calculo_interes,Tasa_interes_prestamo_grande,interes_futuro,Ultimo_interes_pagado,debe_interes,sobrante_abono,Ultimo_abono
0,2624,500.0,40.0,117.5,100.0,False,3,-5,0,924,...,1.5,1,1.5,1,1.5,17.5,60.0,60.0,40.0,100.0
1,2634,400.0,30.0,74.0,60.0,False,4,-3,0,924,...,1.5,1,1.5,1,1.5,14.0,28.0,28.0,22.0,50.0


In [12]:
monitor_grupo.ganancias

,Ganancias_id,Monto_ganancia,Entregada,Socio_id,Sesion_id
0,43464,0.0,0,884,6224
1,43474,0.0,0,894,6224
2,43484,0.0,0,904,6224
3,43494,0.0,0,914,6224
4,43504,0.0,0,924,6224
...,...,...,...,...,...
75,44214,8.8,0,934,6294
76,44224,8.8,0,944,6294
77,44234,8.8,0,954,6294
78,44244,8.8,0,964,6294


In [13]:
monitor_grupo.sesiones

[{'Sesion_id': 2174},
 {'Sesion_id': 6224},
 {'Sesion_id': 6234},
 {'Sesion_id': 6244},
 {'Sesion_id': 6254},
 {'Sesion_id': 6264},
 {'Sesion_id': 6274},
 {'Sesion_id': 6284},
 {'Sesion_id': 6294},
 {'Sesion_id': 6304}]

In [14]:
monitor_grupo.ganancias.groupby('Sesion_id').sum()

,Ganancias_id,Monto_ganancia,Entregada,Socio_id
Sesion_id,,,,
6224,435090,0.0,0,9290
6234,436090,10.0,0,9290
6244,437090,18.0,0,9290
6254,438090,18.0,0,9290
6264,439090,8.0,0,9290
6274,440090,8.0,0,9290
6284,441090,10.0,0,9290
6294,442090,88.0,0,9290
